In [1]:
import pandas as pd
import re
from IPython.display import FileLink
import unicodedata
from sklearn import preprocessing


In [2]:
file_path = r'C:\Users\Βαγγέλης\Desktop\Σχολή\4o Έτος\7ο Εξάμηνο\Ειδικό Θέμα με Τουσίδου (ομαδικό με Σταμάτη)\Datasets\Dataset_6\books.csv'
books_data = pd.read_csv(file_path, on_bad_lines='skip')

In [3]:
print(books_data.columns)

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')


In [4]:
books_data.columns = books_data.columns.str.strip()

books_data.drop(['isbn13', 'text_reviews_count'], axis=1, inplace=True)

books_data.drop_duplicates(inplace=True)

books_data = books_data[books_data['average_rating'] != 0]

In [5]:
Q1 = books_data['average_rating'].quantile(0.25)
Q3 = books_data['average_rating'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 3 * IQR

books_data = books_data[(books_data['average_rating'] >= lower_bound) & (books_data['average_rating'] <= upper_bound)]

In [6]:
unique_languages = books_data['language_code'].unique()
print("Unique values in 'language_code':", unique_languages)

Unique values in 'language_code': ['eng' 'en-US' 'fre' 'spa' 'en-GB' 'mul' 'grc' 'enm' 'en-CA' 'ger' 'jpn'
 'ara' 'nl' 'zho' 'lat' 'por' 'ita' 'rus' 'msa' 'glg' 'wel' 'nor' 'swe'
 'tur' 'gla' 'ale']


In [7]:
books_data['language_code'] = books_data['language_code'].replace(['en-US', 'en-GB', 'en-CA'], 'eng')

In [8]:
unique_languages_updated = books_data['language_code'].unique()
print("Updated unique values in 'language_code':", unique_languages_updated)

Updated unique values in 'language_code': ['eng' 'fre' 'spa' 'mul' 'grc' 'enm' 'ger' 'jpn' 'ara' 'nl' 'zho' 'lat'
 'por' 'ita' 'rus' 'msa' 'glg' 'wel' 'nor' 'swe' 'tur' 'gla' 'ale']


In [9]:
print("Unique values in 'language_code':", books_data['language_code'].unique())

language_counts = books_data['language_code'].value_counts()
print("\nNumber of books per language:\n", language_counts)

non_english_books = books_data[books_data['language_code'] != 'eng']
non_english_count = non_english_books.shape[0]

print(f"\nTotal number of books in languages other than English: {non_english_count}")
print("\nSample of non-English books:\n", non_english_books[['title', 'language_code']].head())

Unique values in 'language_code': ['eng' 'fre' 'spa' 'mul' 'grc' 'enm' 'ger' 'jpn' 'ara' 'nl' 'zho' 'lat'
 'por' 'ita' 'rus' 'msa' 'glg' 'wel' 'nor' 'swe' 'tur' 'gla' 'ale']

Number of books per language:
 language_code
eng    10337
spa      214
fre      143
ger       99
jpn       46
mul       19
zho       14
por       10
grc        9
ita        5
lat        3
enm        3
rus        2
nl         1
ara        1
msa        1
glg        1
wel        1
nor        1
swe        1
tur        1
gla        1
ale        1
Name: count, dtype: int64

Total number of books in languages other than English: 577

Sample of non-English books:
                                            title language_code
50                La Place de la Concorde Suisse           fre
109  Una arruga en el tiempo – A Wrinkle in Time           spa
146                         Cien años de soledad           spa
150           El perfume: Historia de un asesino           spa
201           Timbuktu / Leviathan / Moon Palace 

In [10]:
books_data = books_data[books_data['language_code'].isin(['eng', 'spa', 'fre', 'ger'])]

print("Remaining languages after filtering:\n", books_data['language_code'].value_counts())

Remaining languages after filtering:
 language_code
eng    10337
spa      214
fre      143
ger       99
Name: count, dtype: int64


In [11]:
books_data['language_code'] = books_data['language_code'].replace(['spa', 'fre', 'ger'], 'Other')

print("Updated language counts:\n", books_data['language_code'].value_counts())

Updated language counts:
 language_code
eng      10337
Other      456
Name: count, dtype: int64


In [12]:
special_characters = books_data['title'].str.contains(r'[^\x00-\x7F]', regex=True)
print("Rows with special characters in 'title':", books_data[special_characters])

Rows with special characters in 'title':        bookID                                        title  \
109       201  Una arruga en el tiempo – A Wrinkle in Time   
146       324                         Cien años de soledad   
255       762              Crónica de una muerte anunciada   
256       763                         Cien años de soledad   
268       821       Le Réseau Kinakuta (Cryptonomicon  #2)   
...       ...                                          ...   
11086   45500           La conspiración de los alquimistas   
11094   45531   Montaillou  village occitan de 1294 à 1324   
11107   45592                 La tía Julia y el escribidor   
11108   45595                 La Tía Julia y el Escribidor   
11110   45607                       Las Crónicas de Narnia   

                                                 authors  average_rating  \
109                                    Madeleine L'Engle            4.00   
146                               Gabriel García Márquez      

In [13]:
def remove_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))

In [14]:
def remove_special_characters(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')

books_data['title_normalized'] = books_data['title'].apply(lambda x: remove_special_characters(remove_accents(x)).lower())

In [15]:
print("Title normalization check:\n", books_data[['title', 'title_normalized']].head(10))

Title normalization check:
                                                title  \
0  Harry Potter and the Half-Blood Prince (Harry ...   
1  Harry Potter and the Order of the Phoenix (Har...   
2  Harry Potter and the Chamber of Secrets (Harry...   
3  Harry Potter and the Prisoner of Azkaban (Harr...   
4  Harry Potter Boxed Set  Books 1-5 (Harry Potte...   
5  Unauthorized Harry Potter Book Seven News: "Ha...   
6       Harry Potter Collection (Harry Potter  #1-6)   
7  The Ultimate Hitchhiker's Guide: Five Complete...   
8  The Ultimate Hitchhiker's Guide to the Galaxy ...   
9  The Hitchhiker's Guide to the Galaxy (Hitchhik...   

                                    title_normalized  
0  harry potter and the half-blood prince (harry ...  
1  harry potter and the order of the phoenix (har...  
2  harry potter and the chamber of secrets (harry...  
3  harry potter and the prisoner of azkaban (harr...  
4  harry potter boxed set  books 1-5 (harry potte...  
5  unauthorized harry pot

In [16]:
books_data['title_normalized'] = books_data['title_normalized'].str.lower().apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [17]:
label_encoder = preprocessing.LabelEncoder()
books_data['publisher_encoded'] = label_encoder.fit_transform(books_data['publisher'])

books_data.drop(['publisher'], axis=1, inplace=True)

In [18]:
books_data.drop(['bookID'], axis=1, inplace=True)

In [19]:
C = books_data['average_rating'].mean()

m = books_data['ratings_count'].quantile(0.50)

books_data['weighted_rating'] = (
                                        (books_data['ratings_count'] * books_data['average_rating']) + (m * C)
                                ) / (books_data['ratings_count'] + m)

books_data.drop(['average_rating', 'ratings_count'], axis=1, inplace=True)

print("Updated DataFrame with weighted rating:\n", books_data.head())

Updated DataFrame with weighted rating:
                                                title  \
0  Harry Potter and the Half-Blood Prince (Harry ...   
1  Harry Potter and the Order of the Phoenix (Har...   
2  Harry Potter and the Chamber of Secrets (Harry...   
3  Harry Potter and the Prisoner of Azkaban (Harr...   
4  Harry Potter Boxed Set  Books 1-5 (Harry Potte...   

                      authors        isbn language_code  num_pages  \
0  J.K. Rowling/Mary GrandPré  0439785960           eng        652   
1  J.K. Rowling/Mary GrandPré  0439358078           eng        870   
2                J.K. Rowling  0439554896           eng        352   
3  J.K. Rowling/Mary GrandPré  043965548X           eng        435   
4  J.K. Rowling/Mary GrandPré  0439682584           eng       2690   

  publication_date                                   title_normalized  \
0        9/16/2006  harry potter and the halfblood prince harry po...   
1         9/1/2004  harry potter and the order of the p

In [20]:
print(books_data.columns)

Index(['title', 'authors', 'isbn', 'language_code', 'num_pages',
       'publication_date', 'title_normalized', 'publisher_encoded',
       'weighted_rating'],
      dtype='object')


In [21]:
books_data['weighted_rating'] = books_data['weighted_rating'].round(2)

print("Sample of rounded weighted ratings:\n", books_data[['title', 'weighted_rating']].head())

Sample of rounded weighted ratings:
                                                title  weighted_rating
0  Harry Potter and the Half-Blood Prince (Harry ...             4.57
1  Harry Potter and the Order of the Phoenix (Har...             4.49
2  Harry Potter and the Chamber of Secrets (Harry...             4.37
3  Harry Potter and the Prisoner of Azkaban (Harr...             4.56
4  Harry Potter Boxed Set  Books 1-5 (Harry Potte...             4.76


In [22]:
output_file_path = r'C:\Users\Βαγγέλης\Desktop\cleaned_books_dataset.csv'
books_data.to_csv(output_file_path, index=False, encoding='utf-8')
print(f"DataFrame successfully exported to {output_file_path}")

DataFrame successfully exported to C:\Users\Βαγγέλης\Desktop\cleaned_books_dataset.csv
